# Lab 4: Polymorphisms and Mutations

For lab 4, we will be comparing polymorphisms in two genes, HLA-A and H2A. Part 1 of the lab will involve following a tutorial using the UCSC Genome Browser. Part 2 of the lab will include an ipython lab.

## Importing the Dependencies

In [ ]:
# imports
from datascience import Table
import matplotlib
matplotlib.use('Agg')
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


## Part 1: Exploring polymorphisms in the UCSC Genome Browser

For part 1, we will use the UCSC Genome Browser to explore polymorphisms. The document for part 1 can be found in bcourses. 
Please put your answers for part 1 here:

### Part 1 Answers

#### Questions: Exploring HLA-A
1.

2.

3.

4.

5.

6.

7.


##### Questions: HLA-A SNP Density
1.

#### Questions: HLA-A SNP Search
1.

2.

3.

4.


#### Questions: Exploring H2A
1.

2.

3.

4.

5.


#### Questions: H2A SNP Density
1.

2.


# Part 2: Multiple Sequence Alignment of HLA-A sequences

### Multiple Sequence Alignment of HLA-A sequences

Now, we will align all three HLA-A sequences. This requires a multiple sequence alignment (MSA) algorithm. We have applied an MSA alignment algorithm to these sequences for you. The algorithm we used is called T-coffee, and can be found here: http://tcoffee.vital-it.ch/apps/tcoffee/do:regular Here you can find the alignment between any number of sequences.

## Question 1

Why might a multiple sequence alignment (MSA) algorithm be used instead of a sequence alignment algorithm that just aligns two sequences?


Answer here:

## Question 2

Take a look at the T-coffee website (http://tcoffee.vital-it.ch/apps/tcoffee/references.html) and give a summary about what their algorithm does.


Answer here:

In [ ]:
# load in sequence data for HLA-A
# load in data
table = Table.read_table('https://raw.githubusercontent.com/data-8/mcb-88-connector/gh-pages/data/lab4/hla-a_msa.csv')
table


For each of the aligned sequences, there is either a mismatch between the three sequences, or a dash ('-') that indicates an insertion in that sequence, or a deletion in the other sequences.

Now, we will visualize the sequence differences, by counting the number differences at each location of the three sequences.

First, we define a function, called getDifferenceVector() that returns the counts of differences between three sequences at each location.

In [ ]:
# Function that calculates the number of differences between three sequences. 
# Sequences must be the same length.
def getDifferenceVector(seq1,seq2,seq3):
    if len(seq1) != len(seq2) != len(seq3):
        return('Error: the input strings are not of the same length.')
    # create empty list to hold counts
    counts = [0] * len(seq1)
    index = [0] * len(seq1)
    for i in range(len(seq1)):
        if (seq1[i]!=seq2[i]):
            counts[i]+=1
        if (seq1[i]!=seq3[i]):
            counts[i]+=1
        if (seq2[i]!=seq3[i]):
            counts[i]+=1
        index[i] = i
    return (index, counts)
    

Using this function, we will compute the differences between all three sequences and plot this difference vector.

### Question 3
Fill in the blanks below to plot the difference vector across the three HLA-A sequences.

In [ ]:
# TODO: Select sequences
seq1 = ?
seq2 = ?
seq3 = ?

(index, counts) = getDifferenceVector(seq1,seq2,seq3)
plt.plot(index, counts)
plt.ylabel('Differences')
plt.xlabel('Position')

plt.show()

## Question 4
How does this sample distribution of mismatches compare to the actual distribution for HLA-A we saw in the UCSC Genome Browser?

Answer:

## Question 5

Based on the distribution of mismatches for these three sequences for HLA-A, how would you expect the mismatch distribution for H2A to look?

# Part 3: Extra for Experts
This section is optional. However, it is highly encouraged if you are comfortable with python.

## Bonus Question 1

Choose a SNP overlapping HLA-A from the UCSC Genome Browser (This can be the same SNP you chose in part 1 of the lab.)
Does this SNP appear in any of the sequences above? How can you verify this?